In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split

model = keras.models.load_model("sudoku2.model")

In [2]:
def solveBoard(board):
    while containsZeros(board):
        
        board = standardizeBoard(board) #standardizes the board for prediction
        
        result = model.predict(board.reshape(1,9,9,1)) #all probabilities for all values (0,9] for every square
        result = result.squeeze() #changing the result from (81,9,1) to (81,9)
        
        pred = np.argmax(result, axis = 1).reshape(9,9) + 1 #outputs a board the cnn thinks is the solution (81 squares containing numbers)
        proba = np.around(np.max(result, axis = 1),4).reshape(9,9) #how sure the neural network is that that value is correct
                
        board = regularBoard(board); #undoes standardization
        
        proba = configureProbabilities(board, proba)
        
        board = updateBoard(board, pred, proba)
        
    return board.reshape(81,1)


# Helper Functions for the solveBoard() method above

# REQUIRES: An unstandardized board
# EFFECTS: standardizes the board to be 0 centered and in range -0.5, 0.5
def standardizeBoard(board):
    return (board/9) - 0.5

# REQUIRES: A standardized board
# EFFECTS: undoes the standardization on the board.
def regularBoard(board):
    return (board+0.5) * 9

# REQUIRES: the board is not standardized
# EFFECTS: returns true if the board contains 0
def containsZeros(board):
    rowInd, colInd, dummy = board.shape
    for row in range(rowInd):
        for col in range(colInd):
            if board[row][col] == 0: #maybe need to change it to board[row][col][0] in Java
                return True;
    return False


# REQUIRES: unstandardized board
# EFFECTS: reduces probability of value on a square to 0 if the square isn't empty, or the value violates a sudoku rule
def configureProbabilities(board, proba):
    rowInd, colInd, dummy = board.shape
    #set probability to 0 if square isn't empty
    for row in range(rowInd):
        for col in range(colInd):
            if board[row][col] != 0:
                proba[row][col] = 0
                
    return proba #!!! Make this a public void method in Java
                
    
#EFFECTS: checks if a board is valid (does not violate any of sudoku's rules)
def validBoard(board,pred,x,y):
    temp = np.copy(board) #create a copy of the board... might need to do this the hard way in Java
    temp[x][y] = pred[x][y]
    if validRows(temp):
        if validCols(temp):
            if validSquares(temp):
                return True
            
    return False



#EFFECTS: checks that all the rows don't contain duplicates
def validRows(board):
    valid = True
    for row in range(9):
        temp = []
        for col in range(9):
            temp.append(board[row][col])
        valid = checkCombination(temp)
        if not valid:
            return False
    return True
    
#checks that there are no duplicates in the 9x1 list of combinations
def checkCombination(combination):
    for i in range(9-1):
        if combination[i] != 0:
            current = combination[i]
            j = i+1
            while j<9:
                if current == combination[j]:
                    return False
                j+=1
    return True

# EFFECTS: checks that all the columns don't contain duplicates
def validCols(board):
    valid = True
    for col in range(9):
        temp = []
        for row in range(9):
            temp.append(board[row][col])
        valid = checkCombination(temp)
        if not valid:
            return False
    return True

# EFFECTS: checks that all sudoku squares don't contain duplicates
def validSquares(board):
    valid = True
    for i in [0,3,6]:
        for j in [0,3,6]:
            row = i
            col = j
            colMax = j+3
            temp = []
            while len(temp) < 9:
                temp.append(board[row][col])
                col += 1
                if col >= colMax:
                    col = j
                    row +=1
            valid = checkCombination(temp)
            if not valid:
                return False
    return True
    

#EFFECTS: Fills in a square in the sudoku board that the ai is most sure has the correct value
#    returns a board of only 9s if the ai cannot solve the puzzle
def updateBoard(board, pred, proba):
    searching = True
    while searching:
        x,y = highestProbaInd(proba) #finds square that the ai is most confident in
        
        if x == -1 or y == -1: #means ai cannot find a valid square
            searching = False;
            
        elif validBoard(board, pred, x, y): #the value the ai wants to put there is valid
            searching = False;
            board[x][y] = pred[x][y]
            return board
        
        else: 
            proba[x][y] = 0;
            
    return np.zeros(81).reshape(9,9,1) + 9 #only returned if board is invalid, return a board fille dwith all 9s.
            
    
# EFFECTS: produces true if the board contains all 0s
def allZeros(board):
    rowInd, colInd, dummy = board.shape
    for row in range(rowInd):
        for col in range(colInd):
            if board[row][col] != 0:
                return False
    return True

# EFFECTS: returns the x and y coordinates of the square that the ai is most confident the predicted value is correct.           
def highestProbaInd(proba):
    if allZeros(proba.reshape(9,9,1)): #invalid board
        return -1,-1
    
    x = 0;
    y = 0;
    rsf = 0;
    rowInd, colInd = proba.shape
    for row in range(rowInd):
        for col in range(colInd):
            if proba[row][col] > rsf:
                rsf = proba[row][col]
                x = row
                y = col
    return x,y

In [3]:
df = pd.read_csv("sudoku.csv")

quizzes = df["quizzes"]
solutions = df["solutions"]
featureData = []
for quiz in quizzes:
    temp = []
    for val in quiz:
        temp.append(int(val))
    temp = np.array(temp).reshape(9,9,1)
    featureData.append(temp)
    
featureData = np.array(featureData);

targetData = []
for solution in solutions:
    temp = []
    for val in solution:
        temp.append(int(val))
    temp = np.array(temp).reshape(81,1)
    targetData.append(temp)
    
targetData = np.array(targetData)

xTrain, xTest, yTrain, yTest = train_test_split(featureData, targetData, test_size = 0.2, random_state = 10)

In [4]:
#REQUIRES: featureTest is an unstandarized 9x9x1 npArray, targetTest is an 81x1 solution set, numSamples > 0
#EFFECTS: Evaluates how accurate the full solver is
def evaluateSolver(featureTest, targetTest, numSamples):
    numCorrect = 0
    for i in range(numSamples):
        temp = solveBoard(featureTest[i])
        if (temp == targetTest[i]).sum() == 81:
            numCorrect += 1
    return numCorrect/numSamples

In [5]:
evaluateSolver(xTest,yTest,1000)

1.0

In [12]:
# REQUIRES: a string 81 characteres long that are all the rows of the sudoku board placed next to each other. Empty
#   squares should hold the value of 0. 
#   E.g: '004300209005009001070060043006002087190007400050083000600000105003508690042910300'
# EFFECTS: Produces a solution board

def temporaryUI(boardString):
    initBoard = []
    
    for val in boardString:
        initBoard.append(int(val))
        
    initBoard = np.array(initBoard).reshape(9,9,1)
    
    solvedBoard = solveBoard(initBoard)
    
    return solvedBoard.reshape(9,9)

In [13]:
#quizzes[0]='004300209005009001070060043006002087190007400050083000600000105003508690042910300'
temporaryUI('004300209005009001070060043006002087190007400050083000600000105003508690042910300')

array([[8., 6., 4., 3., 7., 1., 2., 5., 9.],
       [3., 2., 5., 8., 4., 9., 7., 6., 1.],
       [9., 7., 1., 2., 6., 5., 8., 4., 3.],
       [4., 3., 6., 1., 9., 2., 5., 8., 7.],
       [1., 9., 8., 6., 5., 7., 4., 3., 2.],
       [2., 5., 7., 4., 8., 3., 9., 1., 6.],
       [6., 8., 9., 7., 3., 4., 1., 2., 5.],
       [7., 1., 3., 5., 2., 8., 6., 9., 4.],
       [5., 4., 2., 9., 1., 6., 3., 7., 8.]])

In [14]:
solutions[0]

'864371259325849761971265843436192587198657432257483916689734125713528694542916378'